In [1]:
import netCDF4
import torch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, KFold
import pandas as pd

import numpy as np
from dataset import GeoEra5Dataset, GeoEra5Dataset40, GeoUkesmDataset100, SlpEra5Dataset, GeoUkesmDataset
from sklearn import tree
import matplotlib.pyplot as plt

In [2]:
era5_geo_data = GeoEra5Dataset(prefix="../")
era5_geo_data40 = GeoEra5Dataset40(prefix="../")
era5_slp_data = SlpEra5Dataset(prefix="../")
ukesm_geo_data = GeoUkesmDataset(prefix="../")
ukesm_geo_data100 = GeoUkesmDataset100(prefix="../")

era5_geo_data.data = era5_geo_data.data.reshape(
    len(era5_geo_data.data), int(era5_geo_data.data.size / len(era5_geo_data.data))
)

era5_geo_data40.data = era5_geo_data40.data.reshape(
    len(era5_geo_data40.data), int(era5_geo_data40.data.size / len(era5_geo_data40.data))
)

era5_slp_data.data = era5_slp_data.data.reshape(
    len(era5_slp_data.data), int(era5_slp_data.data.size / len(era5_slp_data.data))
)

ukesm_geo_data.data = ukesm_geo_data.data.reshape(
    len(ukesm_geo_data.data), int(ukesm_geo_data.data.size / len(ukesm_geo_data.data))
)

ukesm_geo_data100.data = ukesm_geo_data100.data.reshape(
    len(ukesm_geo_data100.data), int(ukesm_geo_data100.data.size / len(ukesm_geo_data100.data))
)

print("shape of data")
print("era5_geo: " + str(era5_geo_data.data.shape))
print("era5_geo40: " + str(era5_geo_data40.data.shape))
print("era5_slp: " + str(era5_slp_data.data.shape))
print("ukesm_geo: " + str(ukesm_geo_data.data.shape))
print("ukesm_geo100: " + str(ukesm_geo_data100.data.shape))

shape of data
era5_geo: (4018, 22500)
era5_geo40: (3920, 22500)
era5_slp: (4018, 22500)
ukesm_geo: (9898, 22500)
ukesm_geo100: (9800, 22500)


In [7]:
rf = RandomForestClassifier(verbose=1)

param_grid = {
    "n_estimators": [100, 200, 400],
    "max_depth": [None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4, 8],
    "bootstrap": [True],
    "max_features": ["log2"],
}

rf_classifier = RandomForestClassifier()

grid_search = GridSearchCV(
    estimator=rf_classifier, param_grid=param_grid, cv=10, scoring=["f1", "precision", "recall"], refit="f1", n_jobs=-1
)

### era5 hyperparameter search with gridsearch cv
first 3 rows will be printed after finished training. the whole result can be downloaded in the next cell

In [4]:
grid_search.fit(era5_geo_data.data, era5_geo_data.labels)
print(pd.DataFrame(grid_search.cv_results_).sort_values("rank_test_f1").head(3).loc[:, ["params", "mean_test_f1", "rank_test_f1"]])

                                               params  mean_test_f1  \
23  {'bootstrap': True, 'max_depth': None, 'max_fe...      0.783807   
10  {'bootstrap': True, 'max_depth': None, 'max_fe...      0.783030   
21  {'bootstrap': True, 'max_depth': None, 'max_fe...      0.781813   

    rank_test_f1  
23             1  
10             2  
21             3  


In [5]:
pd.DataFrame(grid_search.cv_results_).to_csv('grid_search_results.csv', index=False)

### ukesm hyperparameter search with gridsearch cv
first 3 rows will be printed after finished training. the whole result can be downloaded in the next cell

In [8]:
grid_search.fit(ukesm_geo_data.data, ukesm_geo_data.labels)
print(pd.DataFrame(grid_search.cv_results_).sort_values("rank_test_f1").head(3).loc[:, ["params", "mean_test_f1", "rank_test_f1"]])

                                               params  mean_test_f1  \
8   {'bootstrap': True, 'max_depth': None, 'max_fe...      0.804205   
23  {'bootstrap': True, 'max_depth': None, 'max_fe...      0.802965   
14  {'bootstrap': True, 'max_depth': None, 'max_fe...      0.802482   

    rank_test_f1  
8              1  
23             2  
14             3  


In [9]:
pd.DataFrame(grid_search.cv_results_).to_csv('grid_search_results.csv', index=False)

## final training for era5
first cell is training a yearly split with 41 iterations on the era5 dataset. in the second cell the transferability is tested with a 41 year training dataset on era5 data and a full 101 year test run on ukesm data

In [ ]:
final_param_grid = {
    'bootstrap': True, 
    'max_depth': None, 
    'max_features': 'log2', 
    'min_samples_leaf': 4, 
    'min_samples_split': 5, 
    'n_estimators': 400
}

folds = 41
kf = KFold(n_splits=folds, shuffle=False)

full_labels = []
full_predictions = []

for fold, (train_index, test_index) in enumerate(kf.split(era5_geo_data.data)):    
    train_dataset = era5_geo_data[train_index]
    test_dataset = era5_geo_data[test_index]

    rf = RandomForestClassifier(**final_param_grid)
    rf.fit(train_dataset[0], train_dataset[1])

    test_predictions = rf.predict(test_dataset[0])
    full_predictions.extend(test_predictions)
    full_labels.extend(test_dataset[1])

    if fold % 5 == 0:
        print(f"on fold {fold+1} / {folds}")
        print(f"fold {fold} f1: {f1_score(test_dataset[1], test_predictions)}")

full_predictions = torch.tensor(full_predictions).long()

print("FINAL METRICS")
print(f"f1: {f1_score(full_labels, full_predictions)}")
print(f"precision: {precision_score(full_labels, full_predictions)}")
print(f"recall: {recall_score(full_labels, full_predictions)}")
print(torch.bincount(full_labels))
print(torch.bincount(full_predictions))
print(f"percentage blocked: {(torch.bincount(full_predictions)[1] / len(full_predictions)) * 100}%")

In [ ]:
train_dataset = era5_geo_data
test_dataset = ukesm_geo_data

transfer_rf = RandomForestClassifier(**final_param_grid)
transfer_rf.fit(train_dataset.data, train_dataset.labels)

test_predictions = torch.tensor(rf.predict(test_dataset.data)).long()

print("FINAL METRICS")
print(f"f1: {f1_score(test_dataset.labels, test_predictions)}")
print(f"precision: {precision_score(test_dataset.labels, test_predictions)}")
print(f"recall: {recall_score(test_dataset.labels, test_predictions)}")
print(f"percentage blocked: {(torch.bincount(test_predictions)[1] / len(test_predictions)) * 100}%")

## final training for ukesm

In [ ]:
final_param_grid = {
    'bootstrap': True, 
    'max_depth': None, 
    'max_features': 'log2', 
    'min_samples_leaf': 1, 
    'min_samples_split': 10, 
    'n_estimators': 400
}

folds = 101
kf = KFold(n_splits=folds, shuffle=False)

full_labels = []
full_predictions = []

for fold, (train_index, test_index) in enumerate(kf.split(ukesm_geo_data.data)):    
    train_dataset = ukesm_geo_data[train_index]
    test_dataset = ukesm_geo_data[test_index]

    rf = RandomForestClassifier(**final_param_grid)
    rf.fit(train_dataset[0], train_dataset[1])

    test_predictions = rf.predict(test_dataset[0])
    full_predictions.extend(test_predictions)
    full_labels.extend(test_dataset[1])

    if fold % 10 == 0:
        print(f"on fold {fold+1} / {folds}")
        print(f"fold {fold} f1: {f1_score(test_dataset[1], test_predictions)}")

full_predictions = torch.tensor(full_predictions).long()

print("FINAL METRICS")
print(f"f1: {f1_score(full_labels, full_predictions)}")
print(f"precision: {precision_score(full_labels, full_predictions)}")
print(f"recall: {recall_score(full_labels, full_predictions)}")
print(f"percentage blocked: {(torch.bincount(full_predictions)[1] / len(full_predictions)) * 100}%")

In [ ]:
train_dataset = ukesm_geo_data
test_dataset = era5_geo_data

transfer_rf = RandomForestClassifier(**final_param_grid)
transfer_rf.fit(train_dataset.data, train_dataset.labels)

test_predictions = torch.tensor(rf.predict(test_dataset.data)).long()

print("FINAL METRICS")
print(f"f1: {f1_score(test_dataset.labels, test_predictions)}")
print(f"precision: {precision_score(test_dataset.labels, test_predictions)}")
print(f"recall: {recall_score(test_dataset.labels, test_predictions)}")
print(f"percentage blocked: {(torch.bincount(test_predictions)[1] / len(test_predictions)) * 100}%")

## visualizations for random forest algorithms
set random forest to visualize in the next cell

In [88]:
import datetime
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import confusion_matrix
from util import get_date
import cartopy.crs as ccrs

writer = SummaryWriter(f"../runs_random_forest/{datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')}")
random_forest = transfer_rf

In [89]:
def get_image(data, time):
    fig, axs = plt.subplots(
        nrows=5, ncols=1, subplot_kw={"projection": ccrs.PlateCarree()}
    )

    axs = axs.flatten()
    clevs = np.arange(-1, 3, 0.5)
    long = np.arange(-45, 55, 1)
    lat = np.arange(30, 75, 1)

    for i in range(5):
        time = time + datetime.timedelta(days=1)
        axs[i].coastlines(resolution="110m", linewidth=1)
        cs = axs[i].contourf(
            long,
            lat,
            data[i],
            clevs,
            transform=ccrs.PlateCarree(),
            cmap=plt.cm.RdBu_r,
        )
        if i == 0:
            axs[i].set_title(str(time))

    cbar_ax = fig.add_axes([0.55, 0.15, 0.05, 0.7])
    fig.colorbar(cs, cax=cbar_ax, orientation="vertical")

    plt.draw()

    fig_np = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    fig_np = fig_np.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    fig_np = fig_np.transpose((2, 0, 1))

    plt.close(fig)

    return fig_np

In [ ]:
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=800)
tree.plot_tree(
    transfer_rf.estimators_[0],
    feature_names = np.arange(22500), 
    filled = True,
    max_depth=3,
    fontsize=3
);
fig.savefig('rf_individualtree.png')

In [90]:
ukesm_predictions = random_forest.predict(ukesm_geo_data.data)
era5_predictions = random_forest.predict(era5_geo_data.data)

In [91]:
predictions = ukesm_predictions
dataset = ukesm_geo_data
dataset_type = "ukesm"

conf_matrix = confusion_matrix(dataset.labels, predictions)

false_positives = (predictions == 1) & (dataset.labels == 0)
false_negatives = (predictions == 0) & (dataset.labels == 1)

print("false_positives: " + str(conf_matrix[0][1]))
print("false_negatives: " + str(conf_matrix[1][0]))

for idx, (fp, fn) in enumerate(zip(false_positives, false_negatives)):
    date = get_date(dataset.time[idx], dataset_type)
    if fp.item():
        image = get_image(dataset.data[idx].reshape((5, 45, 100)), date)
        writer.add_image(f"false-positive/rf_{date.strftime('%Y-%m-%d')}", image)
    if fn.item():
        image = get_image(dataset.data[idx].reshape((5, 45, 100)), date)
        writer.add_image(f"false-negative/rf_{date.strftime('%Y-%m-%d')}", image)

false_positives: 4
false_negatives: 29
